# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from geoviews import opts

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,18.53,91,76,2.22,AU,1736703449
1,1,grytviken,-54.2811,-36.5092,8.19,64,81,7.32,GS,1736703450
2,2,isafjordur,66.0755,-23.1240,3.02,93,100,1.54,IS,1736703451
3,3,shimoda,34.6667,138.9500,5.47,71,100,3.34,JP,1736703452
4,4,ilulissat,69.2167,-51.1000,-31.55,98,83,2.91,GL,1736703453


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,      
    size='Humidity',  
    alpha=0.6,        
    tiles='OSM',    
    frame_width=800,
    frame_height=600,
    title="City Humidity Map"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 3) &
     (city_data_df["Max Temp"] <= 27) 
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
print(f"Number of cities matching ideal weather conditions: {len(ideal_weather_df)}")
ideal_weather_df.head()

Number of cities matching ideal weather conditions: 24


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,newman,37.3138,-121.0208,8.35,58,0,2.24,US,1736703349
56,56,palmer,42.1584,-72.3287,2.94,53,0,2.06,US,1736703372
64,64,chanderi,24.7167,78.1333,12.58,61,0,1.91,IN,1736703522
94,94,holualoa,19.6228,-155.9522,17.53,82,0,2.06,US,1736703362
110,110,stanley,54.8680,-1.6985,0.17,95,0,2.06,GB,1736703577


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
47,newman,US,37.3138,-121.0208,58,
56,palmer,US,42.1584,-72.3287,53,
64,chanderi,IN,24.7167,78.1333,61,
94,holualoa,US,19.6228,-155.9522,82,
110,stanley,GB,54.8680,-1.6985,95,
192,brandon,US,27.9378,-82.2859,57,
253,kurara,IN,25.9833,79.9833,65,
281,guerrero negro,MX,27.9769,-114.0611,30,
290,porto-vecchio,FR,41.5910,9.2795,37,
292,yinchuan,CN,38.4681,106.2731,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000 
params = {
        "categories": "accommodation.hotel", 
        "limit": 1, 
        "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"  


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)


    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
chanderi - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
stanley - nearest hotel: Hotel 52
brandon - nearest hotel: Brandon Motor Lodge
kurara - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
porto-vecchio - nearest hotel: No hotel found
yinchuan - nearest hotel: Yinchuan hotel
sur - nearest hotel: Sur Hotel
mount pleasant - nearest hotel: Old Village Post House
fort payne - nearest hotel: Holiday Inn Express & Suites Fort Payne
soloma - nearest hotel: Hotel Terrazas Del Mirador
laguna - nearest hotel: Holiday Inn Express & Suites
oltinko'l - nearest hotel: No hotel found
kittila - nearest hotel: Hotelli Kittilä
kailua-kona - nearest hotel: PACIFIC 19 Kona
jieyang - nearest hotel: 榕江大酒店
fort bragg - nearest hotel: Airborne Inn Lodging
dayrut - nearest hotel: No hotel found
shenjiamen - nearest hotel: Zhonghan Hotel
al jawf - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
47,newman,US,37.3138,-121.0208,58,No hotel found
56,palmer,US,42.1584,-72.3287,53,No hotel found
64,chanderi,IN,24.7167,78.1333,61,No hotel found
94,holualoa,US,19.6228,-155.9522,82,Kona Hotel
110,stanley,GB,54.8680,-1.6985,95,Hotel 52
192,brandon,US,27.9378,-82.2859,57,Brandon Motor Lodge
253,kurara,IN,25.9833,79.9833,65,No hotel found
281,guerrero negro,MX,27.9769,-114.0611,30,Plaza sal paraiso
290,porto-vecchio,FR,41.5910,9.2795,37,No hotel found
292,yinchuan,CN,38.4681,106.2731,40,Yinchuan hotel


In [39]:
name_address 

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': '7天梅州嘉得利店',
    'country': 'China',
    'country_code': 'cn',
    'state': 'Guangdong Province',
    'city': 'Meizhou',
    'town': 'Xincheng Subdistrict',
    'postcode': '514021',
    'district': 'Meixian District',
    'suburb': '程江社区',
    'street': '嘉应西路',
    'lon': 116.10746609999998,
    'lat': 24.298807500000002,
    'formatted': '7 day inn, 嘉应西路, 程江社区, 514021 Guangdong Province, China',
    'address_line1': '7 day inn',
    'address_line2': '嘉应西路, 程江社区, 514021 Guangdong Province, China',
    'categories': ['accommodation',
     'accommodation.hotel',
     'building',
     'building.accommodation'],
    'details': ['details', 'details.contact'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': '7天梅州嘉得利店',
      'phone': '

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot for hotels
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and Latitude for coordinates
    geo=True,      # Enable geographic plotting
    size=100,       # Set point size
    color='Humidity',  # Color points by Humidity
    cmap='viridis',  # Color map for Humidity
    alpha=0.6,         # Transparency of points
    tiles='OSM',       # Base map tiles (OpenStreetMap)
    frame_width=800,
    frame_height=600,
    title="Hotel Map with Hover Information",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]  # Include hover info
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)